# Simulator
Generate simulate PPI network using **forward DMC** model given `q_con`, `q_mod` and graph size `n_nodes`

In [1]:
import networkx as nx
import random
import pathlib

In [1]:
def forward_DMC(G, q_mod, q_con):
    assert(q_mod < 1 and q_mod > 0)
    assert(q_con < 1 and q_con > 0)
    #select anchor node
    nodes = list(G.nodes())
    newnode = sorted(nodes)[-1] + 1
    while(True):
        #duplicate node and edges
        H = G.copy()
        # add in new node
        H.add_node(newnode)

        # randomly choose one node
        anchor = random.choice(nodes)
        # connect the new node with all anchor node's neighbors
        for x in list(G.neighbors(anchor)):
            H.add_edge(newnode, x)
        #remove edges
        for x in list(H.neighbors(newnode)):
            one = random.choice([newnode, anchor])
            if random.random() < q_mod:
                H.remove_edge(one, x)
        #add edge to anchor
        if random.random() < q_con:
            H.add_edge(anchor, newnode)
        ##############KEEPING ONLY CONNECTED GRAPHS CHANGE LATER#########################
        if (nx.is_connected(H)):
            break
    # add in the final anchor new node pair
    return H, (anchor, newnode)

In [2]:
def simulate_evolution(num_nodes, q_mod, q_con):    
    G1=nx.Graph()
    G1.add_node(1)
    G1.add_node(2)
    G1.add_edge(1,2)
    graph_list = []
    graph_list_pairs = []
    pair_list = []
    #print('G #1 created')
    #print(G1.edges())
    G = G1
    for i in range(3, num_nodes+1):
        #print('Creating G#', i-1)
        G, pair = forward_DMC(G, q_mod, q_con)
        graph_list.append(G)
        pair_list.append(pair)
        graph_list_pairs.append([])

        for x,y in G.edges():
            graph_list_pairs[-1].append((x,y))

    simulation = {'graph_list': graph_list_pairs, 'anchor_dup_pair': pair_list}    
    return graph_list, pair_list, simulation

In [3]:
def write_graph(filename, graph):
    with open(filename, 'w') as f:
        for x,y in graph.edges():
            f.write(str(x)+' '+str(y)+'\n')

In [5]:
# number of nodes within PPI network
n_nodes = 50

# q_mod parameter in DMC model
q_mod = 0.7

# q_con parameter in DMC model
q_con = 0.4

# start simulation
graph_list, pair_list, simulation = simulate_evolution(num_nodes = n_nodes, q_mod = q_mod, q_con = q_con)

# write result
write_graph('./Example_data/nG' + str(n_nodes) + '/nx_run=1_qmod='+str(q_mod)+'_qcon='+str(q_con)+'_n='+str(n_nodes), graph_list[-1])
with open('./Example_data/nG' + str(n_nodes) + '/ground_truth_qmod=' + str(q_mod) + '_qcon=' +str(q_con),'w') as f:
    for x,y in pair_list:
        f.write(str(x) + ' '+str(y) +'\n')